<a href="https://colab.research.google.com/github/coderanandmaurya/PySpark/blob/main/PySpark_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

In [2]:
ps=SparkSession.builder.appName('day2').getOrCreate()

In [3]:
ps

In [5]:
df=ps.read.csv('/content/Day2_Spark_Data.csv',header=True,inferSchema=True)

In [6]:
df.show()

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|sahil|   9|  10|    BCA|
| aman|   8|   9|   NULL|
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
|mohan|NULL|   6|B.Tech.|
+-----+----+----+-------+



In [7]:
#drop col
df.drop('Name').show()

+----+----+-------+
|CGPA| PKG| course|
+----+----+-------+
|   9|  10|    BCA|
|   8|   9|   NULL|
|   6|   6|B.Tech.|
|   7|NULL|    BCA|
|NULL|   6|B.Tech.|
+----+----+-------+



In [8]:
df.show()

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|sahil|   9|  10|    BCA|
| aman|   8|   9|   NULL|
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
|mohan|NULL|   6|B.Tech.|
+-----+----+----+-------+



In [11]:
#droping Nan Values
df.na.drop().show()

+-----+----+---+-------+
| NAME|CGPA|PKG| course|
+-----+----+---+-------+
|sahil|   9| 10|    BCA|
|jerry|   6|  6|B.Tech.|
+-----+----+---+-------+



In [12]:
df.na.drop(how='any').show() #any=any null or all all row is null

+-----+----+---+-------+
| NAME|CGPA|PKG| course|
+-----+----+---+-------+
|sahil|   9| 10|    BCA|
|jerry|   6|  6|B.Tech.|
+-----+----+---+-------+



In [13]:
df.na.drop(how='all').show()

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|sahil|   9|  10|    BCA|
| aman|   8|   9|   NULL|
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
|mohan|NULL|   6|B.Tech.|
+-----+----+----+-------+



In [14]:
df.na.drop(thresh=2).show() #atleast 2 non null values

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|sahil|   9|  10|    BCA|
| aman|   8|   9|   NULL|
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
|mohan|NULL|   6|B.Tech.|
+-----+----+----+-------+



In [16]:
#subset
df.na.drop(subset=['CGPA']).show()

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|sahil|   9|  10|    BCA|
| aman|   8|   9|   NULL|
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
+-----+----+----+-------+



In [22]:
# filling Missing Value
df.na.fill('Missing','course').show() #parameter (value,subset)

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|sahil|   9|  10|    BCA|
| aman|   8|   9|Missing|
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
|mohan|NULL|   6|B.Tech.|
+-----+----+----+-------+



In [24]:
#fill nan value in PKG
df.na.fill(0,'PKG').show()

+-----+----+---+-------+
| NAME|CGPA|PKG| course|
+-----+----+---+-------+
|sahil|   9| 10|    BCA|
| aman|   8|  9|   NULL|
|jerry|   6|  6|B.Tech.|
|raman|   7|  0|    BCA|
|mohan|NULL|  6|B.Tech.|
+-----+----+---+-------+



In [25]:
#using Simple Imputer
from pyspark.ml.feature import Imputer

Imputer=Imputer(
    inputCols=['PKG','CGPA'],
    outputCols=["{}_imputed".format(c) for c in ['PKG','CGPA']]
).setStrategy('mean')  #mean,median

In [26]:
Imputer.fit(df).transform(df).show()

+-----+----+----+-------+-----------+------------+
| NAME|CGPA| PKG| course|PKG_imputed|CGPA_imputed|
+-----+----+----+-------+-----------+------------+
|sahil|   9|  10|    BCA|         10|           9|
| aman|   8|   9|   NULL|          9|           8|
|jerry|   6|   6|B.Tech.|          6|           6|
|raman|   7|NULL|    BCA|          7|           7|
|mohan|NULL|   6|B.Tech.|          6|           7|
+-----+----+----+-------+-----------+------------+



In [30]:
#Filter Operation
# <,>&,|,==,~
#student having cgpa 8 or higher having
df.filter('CGPA>=8').show()

+-----+----+---+------+
| NAME|CGPA|PKG|course|
+-----+----+---+------+
|sahil|   9| 10|   BCA|
| aman|   8|  9|  NULL|
+-----+----+---+------+



In [32]:
# for specific col name, CGPA
df.filter('CGPA>=8').select(['Name','CGPA']).show()

+-----+----+
| Name|CGPA|
+-----+----+
|sahil|   9|
| aman|   8|
+-----+----+



In [33]:
# two condition
df.filter((df['CGPA']>=8) & (df['PKG']>8)).show()

+-----+----+---+------+
| NAME|CGPA|PKG|course|
+-----+----+---+------+
|sahil|   9| 10|   BCA|
| aman|   8|  9|  NULL|
+-----+----+---+------+



In [34]:
# inverse condition
df.filter(~(df['CGPA']>=8)).show()

+-----+----+----+-------+
| NAME|CGPA| PKG| course|
+-----+----+----+-------+
|jerry|   6|   6|B.Tech.|
|raman|   7|NULL|    BCA|
+-----+----+----+-------+



In [43]:
# Group by and aggregate function
df.groupBy('course').sum().show()

+-------+---------+--------+
| course|sum(CGPA)|sum(PKG)|
+-------+---------+--------+
|   NULL|        8|       9|
|B.Tech.|        6|      12|
|    BCA|       16|      10|
+-------+---------+--------+



In [36]:
df.groupBy('Name')

GroupedData[grouping expressions: [Name], value: [NAME: string, CGPA: int ... 2 more fields], type: GroupBy]

In [39]:
df.groupBy('Name').sum().show()

+-----+---------+--------+
| Name|sum(CGPA)|sum(PKG)|
+-----+---------+--------+
|mohan|     NULL|       6|
|jerry|        6|       6|
|raman|        7|    NULL|
|sahil|        9|      10|
| aman|        8|       9|
+-----+---------+--------+



In [41]:
# group by course having max PKG
df.groupBy('course').max().show()

+-------+---------+--------+
| course|max(CGPA)|max(PKG)|
+-------+---------+--------+
|   NULL|        8|       9|
|B.Tech.|        6|       6|
|    BCA|        9|      10|
+-------+---------+--------+



In [44]:
# mean of pkg
df.groupBy('course').mean().show()

+-------+---------+--------+
| course|avg(CGPA)|avg(PKG)|
+-------+---------+--------+
|   NULL|      8.0|     9.0|
|B.Tech.|      6.0|     6.0|
|    BCA|      8.0|    10.0|
+-------+---------+--------+



In [45]:
# count
df.groupBy('course').count().show()

+-------+-----+
| course|count|
+-------+-----+
|   NULL|    1|
|B.Tech.|    2|
|    BCA|    2|
+-------+-----+



In [46]:
df.agg({'PKG':'sum'}).show()

+--------+
|sum(PKG)|
+--------+
|      31|
+--------+

